# Demonstrate Seq2Seq Wrapper with Cornell Movie Dialog Corpus

In [7]:
import tensorflow as tf
import numpy as np

# preprocessed data
from datasets.cornell_corpus import data
import data_utils

In [8]:
import importlib
importlib.reload(data)

<module 'datasets.cornell_corpus.data' from '/Users/unclewang/Documents/GitHub/practical_seq2seq/datasets/cornell_corpus/data.py'>

In [9]:
# load data from pickle and npy files
metadata, idx_q, idx_a = data.load_data(PATH='datasets/cornell_corpus/')
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(idx_q, idx_a)

In [10]:
# parameters 
xseq_len = trainX.shape[-1]
yseq_len = trainY.shape[-1]
batch_size = 16
xvocab_size = len(metadata['idx2w'])  
yvocab_size = xvocab_size
emb_dim = 1024

In [11]:
import seq2seq_wrapper

In [12]:
model = seq2seq_wrapper.Seq2Seq(xseq_len=xseq_len,
                               yseq_len=yseq_len,
                               xvocab_size=xvocab_size,
                               yvocab_size=yvocab_size,
                               ckpt_path='ckpt/cornell_corpus/',
                               emb_dim=emb_dim,
                               num_layers=3
                               )

<log> Building Graph 

TypeError: can't pickle _thread.lock objects

In [7]:
val_batch_gen = data_utils.rand_batch_gen(validX, validY, 32)
test_batch_gen = data_utils.rand_batch_gen(testX, testY, 256)
train_batch_gen = data_utils.rand_batch_gen(trainX, trainY, batch_size)

In [8]:
sess = model.restore_last_session()

In [16]:
input_ = test_batch_gen.__next__()[0]
output = model.predict(sess, input_)
print(output.shape)

(256, 25)


In [17]:
replies = []
for ii, oi in zip(input_.T, output):
    q = data_utils.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded = data_utils.decode(sequence=oi, lookup=metadata['idx2w'], separator=' ').split(' ')
    if decoded.count('unk') == 0:
        if decoded not in replies:
            print('q : [{0}]; a : [{1}]'.format(q, ' '.join(decoded)))
            replies.append(decoded)

q : [bill id like you to meet jack torrance]; a : [how much do you think]
q : [and who are you to talk you were nothing you couldnt even sing i must have been out of my mind]; a : [i dont know what youre talking about]
q : [by breaking up a companys assets]; a : [what are you talking about]
q : [what is it]; a : [i dont know]
q : [ill see you there]; a : [ill get out]
q : [okay ill be talking to you]; a : [youre not going to get out]
q : [i must be outta my mind buddy quit it]; a : [okay for a minute]
q : [when are you going to let the police know]; a : [you dont know what youre talking about]
q : [you can do it]; a : [yeah i think so]
q : [like hell you know if you fellows stuck together stead of letting them walk all over you they might not try it]; a : [if you werent talking about me i dont know what youre talking about]
q : [wait are you saying that i dont appreciate]; a : [i know you know what i mean]
q : [no you just looked as if you did]; a : [well i didnt believe you]
q : [your